In [1]:
import camelot
import numpy as np
import pandas as pd

In [43]:
file = "Bank_Statements/sbi2bs.pdf"

In [44]:
import pdfplumber

with pdfplumber.open(file) as pdf:
    
    total_pages = pdf.pages
total = len(total_pages)
total

8

In [45]:
tables = camelot.read_pdf(file,flavor='lattice')
tables[0].df

,0,1,2,3,4,5,6
0,Txn Date,Value\nDate,Description,Ref No./Cheque\nNo.,Debit,Credit,Balance
1,1 Nov 2018,1 Nov 2018 TO TRANSFER-,UPI/DR/830500706577/311933\n81/sbin/3119338109-,,"2,000.00",,"7,309.24"
2,1 Nov 2018,1 Nov 2018 TO TRANSFER-,UPI/DR/830521870206/billdesk/\nICIC/billdesk.a-,,150.00,,"7,159.24"
3,2 Nov 2018,2 Nov 2018 TO TRANSFER-,UPI/DR/830613209219/733022\n10/synb/7330221000-,,300.00,,"6,859.24"
4,2 Nov 2018,2 Nov 2018 TO TRANSFER-,UPI/DR/830616671072/733010\n10/synb/7330101001-,,"4,000.00",,"2,859.24"
5,3 Nov 2018,,3 Nov 2018 CREDIT- REJ ON 28/10/2018-,,,"19,300.00","22,159.24"
6,3 Nov 2018,,3 Nov 2018 CREDIT- REJ ON 28/10/2018-,,,200.00,"22,359.24"
7,3 Nov 2018,3 Nov 2018 TO TRANSFER-,UPI/DR/830723471560/LABAJI\nT /SBIN/labajitsar-,,"3,000.00",,"19,359.24"
8,3 Nov 2018,3 Nov 2018 TO TRANSFER-,UPI/DR/830723474608/LABAJI\nT /SBIN/labajitsar-,,500.00,,"18,859.24"
9,3 Nov 2018,3 Nov 2018 BY TRANSFER-,UPI/CR/830723473488/LABAJI\nT /SBIN/labajitsar-,,,500.00,"19,359.24"


In [46]:
def processing(data):
    headers = data.iloc[0]
    data  = pd.DataFrame(data.values[1:], columns=headers)
    data = data.drop(columns=['Ref No./Cheque\nNo.'])

    data = data.replace(r'^\s*$', np.nan, regex=True)
    data['Debit'] = data['Debit'].str.replace(',', '').astype(float)
    data['Credit'] = data['Credit'].str.replace(',', '').astype(float)
    data['Balance'] = data['Balance'].str.replace(',', '').astype(float)

    return data

In [47]:
df = pd.DataFrame()
for i in range(1,total+1):
    
    print(i)
    tables = camelot.read_pdf(file,flavor='lattice',pages=str(i))
    print(tables)
    if len(tables)!=0:
        
        df1 = processing(tables[0].df)

        df = pd.concat([df, df1])
df = df.fillna(0)
df.index = range(df.shape[0])

1
<TableList n=1>
2
<TableList n=1>
3
<TableList n=1>
4
<TableList n=1>
5
<TableList n=1>
6
<TableList n=1>
7
<TableList n=1>
8
<TableList n=0>


In [48]:
df

,Txn Date,Value\nDate,Description,Debit,Credit,Balance
0,1 Nov 2018,1 Nov 2018 TO TRANSFER-,UPI/DR/830500706577/311933\n81/sbin/3119338109-,2000.0,0.0,7309.24
1,1 Nov 2018,1 Nov 2018 TO TRANSFER-,UPI/DR/830521870206/billdesk/\nICIC/billdesk.a-,150.0,0.0,7159.24
2,2 Nov 2018,2 Nov 2018 TO TRANSFER-,UPI/DR/830613209219/733022\n10/synb/7330221000-,300.0,0.0,6859.24
3,2 Nov 2018,2 Nov 2018 TO TRANSFER-,UPI/DR/830616671072/733010\n10/synb/7330101001-,4000.0,0.0,2859.24
4,3 Nov 2018,0,3 Nov 2018 CREDIT- REJ ON 28/10/2018-,0.0,19300.0,22159.24
...,...,...,...,...,...,...
143,19 Jan\n2019,19 Jan\n2019,TO TRANSFER-\nUPI/DR/901913897883/733120\n10/s...,5000.0,0.0,2004.48
144,21 Jan\n2019,21 Jan\n2019,DEBIT-ACHDr\nCITI00002000000037 BD-\nBIRLA MF OT-,2000.0,0.0,4.48
145,22 Jan\n2019,22 Jan\n2019,BY TRANSFER-INB\nIMPS902212496204/98594661\n87...,0.0,8000.0,8004.48
146,22 Jan\n2019,22 Jan\n2019,TO TRANSFER-INB\nIMPS/P2A/902212700249/XXX\nXX...,5000.0,0.0,3004.48


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Txn Date     148 non-null    object 
 1   Value
Date   148 non-null    object 
 2   Description  148 non-null    object 
 3   Debit        148 non-null    float64
 4   Credit       148 non-null    float64
 5   Balance      148 non-null    float64
dtypes: float64(3), object(3)
memory usage: 7.1+ KB
